In [1]:
import os

In [2]:
%pwd

'd:\\CV-Projects\\End-to-End-Chest-Cancer-Classification-MLflow-DVC\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\CV-Projects\\End-to-End-Chest-Cancer-Classification-MLflow-DVC'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/atharvac1301/End-to-End-Chest-Cancer-Classification-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="atharvac1301"
os.environ["MLFLOW_TRACKING_PASSWORD"]="fcc56715d30099c587b900a8711bff8195c6cc1a"


In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    mlflow_uri: str
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [8]:
from cnnClassifier.constants.constant import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        print(type(config_filepath), config_filepath)
        print(type(params_filepath), params_filepath)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/chest-cancer-data",
            mlflow_uri="https://dagshub.com/atharvac1301/End-to-End-Chest-Cancer-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

        )

        return eval_config
    

In [10]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        
        # mlflow.set_registry_uri(self.config.mlflow_uri)
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
        
        

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e 

<class 'pathlib.WindowsPath'> config\config.yaml
<class 'pathlib.WindowsPath'> params.yaml
[2024-03-05 02:39:48,830: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-05 02:39:48,832: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 02:39:48,834: INFO: common: created directory at: artifacts]
Found 52 images belonging to 2 classes.

[2024-03-05 02:39:55,001: WARNING: module_wrapper: From e:\Anaconda\anaconda_setup\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
4/4 [==============================] - 18s 1s/step - loss: 0.5883 - accuracy: 0.8269
[2024-03-05 02:40:08,283: INFO: common: json file saved at: scores.json]


2024/03/05 02:40:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp58tfqyls\model\data\model\assets
[2024-03-05 02:40:35,026: INFO: builder_impl: Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp58tfqyls\model\data\model\assets]


e:\Anaconda\anaconda_setup\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/03/05 02:41:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 27
Created version '27' of model 'VGG16Model'.
